# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

### Downloading data

In [2]:
! pip install opendatasets

tnnz20Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import opendatasets as od
od.download('https://www.kaggle.com/c/dogs-vs-cats')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: tnnz20
Your Kaggle Key: ··········


100%|██████████| 812M/812M [00:03<00:00, 236MB/s]



Extracting archive ./dogs-vs-cats/dogs-vs-cats.zip to ./dogs-vs-cats


In [4]:
import zipfile
local_train_zip = '/content/dogs-vs-cats/train.zip'

def extract_zip(local_zip):
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/content/')
  zip_ref.close()

extract_zip(local_train_zip)

In [5]:
import os
import shutil


root_train_dir = './Cat_dog_data/train'
root_test_dir = './Cat_dog_data/test'

def create_dirs(root_dir):
# Empty directory to prevent FileExistsError is the function is run several times
  if os.path.exists(root_dir):
    shutil.rmtree(root_dir)

  # Create new folder
  cat_dir = ['cat', 'dog']
  for dir in cat_dir:
    try:
      new_dir = os.path.join(root_dir, dir)
      os.makedirs(new_dir)
      print(f"Directory {new_dir} created successfully")
    except OSError as error:
      print(error)

create_dirs(root_train_dir)
create_dirs(root_test_dir)

Directory ./Cat_dog_data/train/cat created successfully
Directory ./Cat_dog_data/train/dog created successfully
Directory ./Cat_dog_data/test/cat created successfully
Directory ./Cat_dog_data/test/dog created successfully


In [6]:
import random
base_dir = './train'

def copy_file(base_dir, root_dir, dest_dir, type_dir):
  for file_name in dest_dir:
    target_dir = os.path.join(root_dir, type_dir)
    copied_file = os.path.join(base_dir, file_name)
    shutil.copy(copied_file, target_dir)

def split_file(base_dir, split_size):

  temp_cat = []
  temp_dog = []
  for file_name in os.listdir(base_dir):
    split_name = file_name.split('.')[0]
    if split_name == 'cat':
      temp_cat.append(file_name)
    else:
      temp_dog.append(file_name)
  
  # Create split point
  split_point_cat = int(len(temp_cat) * split_size)
  split_point_dog = int(len(temp_dog) * split_size)

  # Shuffle list
  shuffle_list_cat = random.sample(temp_cat, len(temp_cat))
  shuffle_list_dog = random.sample(temp_dog, len(temp_dog))

  # Split 
  train_cat = shuffle_list_cat[:split_point_cat]
  test_cat = shuffle_list_cat[split_point_cat:]

  train_dog = shuffle_list_dog[:split_point_dog]
  test_dog = shuffle_list_dog[split_point_dog:]

  root_dir = './Cat_dog_data/'
  copy_file(base_dir, root_dir, train_cat, type_dir='train/cat')
  copy_file(base_dir, root_dir, test_cat, type_dir='test/cat')
  copy_file(base_dir, root_dir, train_dog, type_dir='train/dog')
  copy_file(base_dir, root_dir, test_dog, type_dir='test/dog')


split_file(base_dir, split_size=0.9)

In [7]:
import pathlib
PATH = './Cat_dog_data'
data_dir = pathlib.Path(PATH)

image_count = len(list(data_dir.glob('*/*/*.jpg')))
print(f'Total image from this datasets : {image_count}')
for i in os.listdir(data_dir):
  image_count = len(list(data_dir.glob(f'{i}/*/*.jpg')))
  print(f'Total image from this {i} : {image_count}')

Total image from this datasets : 25000
Total image from this test : 2500
Total image from this train : 22500


### Notebook

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [9]:
data_dir = './Cat_dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])
                                       ])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])
                                      ])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [10]:
model = models.densenet121(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [11]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [12]:
import time

In [13]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 4.243 seconds
Device = cuda; Time per batch: 0.012 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [15]:
# set up GPU for pytorch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

Download resnet50 as pretrained model

In [16]:
## TODO: Use a pretrained model to classify the cat and dog images
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [17]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
# Turn off gradient for model
for param in model.parameters():
  param.requires_grad = False

# Define new classifier
classifier = nn.Sequential(nn.Linear(2048, 512),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(512, 2),
                           nn.LogSoftmax(dim=1)
                       )

model.fc = classifier

In [19]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [23]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# use GPU 
model.to(device);

In [27]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5

# Train model
for epoch in range(epochs):
  for images, labels in trainloader:
    steps += 1
    
    # set data to use GPU
    images, labels = images.to(device), labels.to(device)

    start = time.time()
    
    # clear gradient
    optimizer.zero_grad()

    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if steps % print_every == 0:
      test_loss = 0
      accuracy = 0

      model.eval()
      with torch.no_grad():
        for images, labels in testloader:
          # set data to use GPU
          images, labels = images.to(device), labels.to(device)

          logps = model(images)
          loss = criterion(logps, labels)
          test_loss += loss.item()

          # calculate accuracy
          ps = torch.exp(logps)
          top_ps, top_class = ps.topk(1, dim=1)
          equality = top_class == labels.view(*top_class.shape)
          accuracy += torch.mean(equality.type(torch.FloatTensor)).item()

      result = f'''Time: {(time.time() - start)/3:.3f}
      Epoch: {epoch+1}/{epochs}..
      Train loss: {running_loss/print_every:.3f}..
      Test loss: {test_loss/len(testloader):.3f}..
      Test accuracy: {accuracy/len(testloader):.3f}
      '''
      print(result)

      running_loss = 0
      model.train()

Time: 7.621
      Epoch: 1/1..
      Train loss: 0.194..
      Test loss: 0.121..
      Test accuracy: 0.946
      
Time: 7.473
      Epoch: 1/1..
      Train loss: 0.139..
      Test loss: 0.121..
      Test accuracy: 0.950
      
Time: 7.410
      Epoch: 1/1..
      Train loss: 0.143..
      Test loss: 0.152..
      Test accuracy: 0.933
      
Time: 7.427
      Epoch: 1/1..
      Train loss: 0.200..
      Test loss: 0.163..
      Test accuracy: 0.920
      
Time: 7.395
      Epoch: 1/1..
      Train loss: 0.137..
      Test loss: 0.129..
      Test accuracy: 0.944
      
Time: 7.579
      Epoch: 1/1..
      Train loss: 0.165..
      Test loss: 0.153..
      Test accuracy: 0.929
      
Time: 7.384
      Epoch: 1/1..
      Train loss: 0.113..
      Test loss: 0.216..
      Test accuracy: 0.904
      
Time: 7.422
      Epoch: 1/1..
      Train loss: 0.273..
      Test loss: 0.190..
      Test accuracy: 0.921
      
Time: 7.431
      Epoch: 1/1..
      Train loss: 0.189..
      Test loss

KeyboardInterrupt: ignored